# CW Attack Test

TJ Kim <br/>
2/2/21

#### Objective: 
Run CW Attack on generic federated learning setting.
See what the transfer rate is.

The goal is to build an infrastructure of C&W Attack Implementation.


In [4]:
cd '/home/ubuntu/FedAtk/' 

/home/ubuntu/FedAtk


### Load Relevant Libraries and Modules

Load the relevant libraries for the federated learning code.

In [5]:
import time
import yaml
        
from femnist_dataloader import Dataloader
from cnn_head import CNN_Head
from cnn_neck import CNN_Neck
from cnn_server import Server
from cnn_client import Client
from data_manager import DataManager

from utilities import freeze_layers
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import csv
import os
import pickle
from torch.autograd import Variable

import multiprocessing as mp

import queue

# Extra not from py file
from collections import OrderedDict 
import itertools

# Import C&W Attack Modules
import cw_attack.cw

# Torch and plotting
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

### Directly Make victim NN Class

In [9]:
class Victim_NN_CW(nn.Module):
    """
    Summary: 
    
    Pytorch NN module that takes pre-trained weights from layered personalized model
    We also load the data-loader and give test,attack functionality
    
    """
    
    def __init__(self, head_network, neck_network, dataloader):
        
        # Init attributes
        super(Victim_NN, self).__init__()
        self.head = head_network
        self.neck = neck_network
        self.dataloader = dataloader
        self.criterion = nn.NLLLoss()
        
        # Image min/max range per channel
        mean = (0.5)
        std = (0.5)

        self.inputs_box = (min((0 - m) / s for m, s in zip(mean, std)),
                           max((1 - m) / s for m, s in zip(mean, std)))
        
        # test_acc attributes
        self.orig_test_acc = None
        self.adv_test_acc = None
        
        self.orig_output_sim = None
        self.adv_output_sim = None
        
        # Attack attributes
        self.x_orig = None
        self.x_adv = None
        self.y_orig = None
        self.target = None
        
        self.softmax_orig = None
        self.output_orig = None
        self.softmax_adv = None
        self.output_adv = None
        
        self.orig_loss = None
        self.adv_loss = None
        self.orig_acc = None
        self.adv_acc = None
        
    def forward(self,x):
        x = self.neck.forward(x)
        x = self.head.forward(x)
        
        return x
    
    def forward_transfer(self, x_orig, x_adv, y_orig, y_adv,
                         true_labels, target, print_info = False):
        """
        Assume that input images are in pytorch tensor format
        """
        
        batch_size = y_orig.shape[0]
        
        # Forward Two Input Types
        h_adv = self.forward(x_adv)
        h_orig = self.forward(x_orig)
        h_adv_category = torch.argmax(h_adv,dim = 1)
        h_orig_category = torch.argmax(h_orig,dim = 1)
        
        # Record Different Parameters
        self.orig_test_acc = (h_orig_category == true_labels).float().sum()/batch_size
        self.adv_test_acc = (h_adv_category == true_labels).float().sum()/batch_size
        
        self.orig_output_sim = (h_orig_category == y_orig).float().sum()/batch_size
        self.adv_output_sim = (h_adv_category == y_adv).float().sum()/batch_size
        
        self.orig_target_achieve = (h_orig_category == target).float().sum()/batch_size
        self.adv_target_achieve = (h_adv_category == target).float().sum()/batch_size

        
        # Print Relevant Information
        if print_info:
            print("---- Attack Transfer:", "----\n")
            print("         Orig Test Acc:", self.orig_test_acc.item())
            print("          Adv Test Acc:", self.adv_test_acc.item())
            print("Orig Output Similarity:", self.orig_output_sim.item())
            print(" Adv Output Similarity:", self.adv_output_sim.item())
            print("       Orig Target Hit:", self.orig_target_achieve.item())
            print("        Adv Target Hit:", self.adv_target_achieve.item())
            
    def specs_from_xadv(self, print_info=False):
        
        self.softmax_orig = self.forward(self.x_orig)
        self.output_orig = torch.argmax(self.softmax_orig,dim=1)
        self.softmax_adv = self.forward(self.x_adv)
        self.output_adv = torch.argmax(self.softmax_adv,dim=1)
        
        # Record accuracy and loss
        self.orig_loss = self.criterion(self.softmax_orig, self.y_orig).item()
        self.adv_loss = self.criterion(self.softmax_adv, self.y_orig).item()
        self.orig_acc = (self.output_orig == self.y_orig).float().sum()/batch_size
        self.adv_acc = (self.output_adv == self.y_orig).float().sum()/batch_size
        
        # Add Perturbation Distance (L2 norm) - across each input
        self.norm = torch.norm(torch.sub(self.x_orig, self.x_adv, alpha=1),dim=(2,3))

        # Print Relevant Information
        if print_info:
            print("---- FGSM Batch Size:", batch_size, "----\n")
            print("Orig Target:", self.y_orig.tolist())
            print("Orig Output:", self.output_orig.tolist())
            print("ADV Output :", self.output_adv.tolist(),'\n')
            print("Orig Loss  :", self.orig_loss)
            print("ADV Loss   :", self.adv_loss,'\n')
            print("Orig Acc   :", self.orig_acc.item())
            print("ADV Acc    :", self.adv_acc.item())
            
    def CW_attack(self, batch_size = 10, target= -1, confidence=0.0, optimizer_lr=5e-4, 
                  iteration=1, x_val_mean = (0.5), x_val_std= (0.5), print_info=False):
        
        image_data = self.dataloader.load_batch(batch_size)
        self.x_orig  = torch.Tensor(image_data['input']).reshape(batch_size,1,28,28)
        self.y_orig = torch.Tensor(image_data['label']).type(torch.LongTensor).cuda()
        self.target = target
        
        if target > -1:
            targeted = False
            targets = self.y_orig
        else:
            targeted = True
            targets = torch.ones(image_data['input'].size(0), dtype = torch.long) * target
            
        
        adversary = cw.L2Adversary(targeted=targeted,
                           confidence=confidence,
                           search_steps=iteration,
                           box=inputs_box,
                           optimizer_lr=optimizer_lr)
        
        self.x_adv = adversary(self, self.x_orig, targets, to_numpy=False)
        assert isinstance(self.x_adv, torch.FloatTensor)
        assert self.x_adv.size() == self.x_orig.size()
        
        self.specs_from_xadv(print_info)
        
        
    def i_fgsm(self, batch_size = 10, target= -1, eps=0.03, alpha=1, 
               iteration=1, x_val_min=-1, x_val_max=1, print_info=False):
        """
        batch_size - number of images to adversarially perturb
        targetted - target class output we desire to alter all inputs into
        eps - max amount to add perturbations per pixel per iteration
        alpha - gradient scaling (increase minimum perturbation amount below epsilon)
        iteration - how many times to perturb
        x_val_min/max - NN input valid range to keep perturbations within
        """
        self.eval()
        
        # Load data to perturb
    
        image_data = self.dataloader.load_batch(batch_size)
        self.x_orig  = torch.Tensor(image_data['input']).reshape(batch_size,1,28,28)
        self.y_orig = torch.Tensor(image_data['label']).type(torch.LongTensor).cuda()
        self.target = target
        
        self.x_adv = Variable(self.x_orig, requires_grad=True)
        
        for i in range(iteration):
            
            h_adv = self.forward(self.x_adv)
            
            # Loss function based on target
            if target > -1:
                target_tensor = torch.LongTensor(self.y_orig.size()).fill_(target)
                target_tensor = Variable(cuda(target_tensor, self.cuda), requires_grad=False)
                cost = self.criterion(h_adv, target_tensor)
            else:
                cost = -self.criterion(h_adv, self.y_orig)

            self.zero_grad()

            if self.x_adv.grad is not None:
                self.x_adv.grad.data.fill_(0)
            cost.backward()

            self.x_adv.grad.sign_()
            self.x_adv = self.x_adv - alpha*self.x_adv.grad
            self.x_adv = where(self.x_adv > self.x_orig+eps, self.x_orig+eps, self.x_adv)
            self.x_adv = where(self.x_adv < self.x_orig-eps, self.x_orig-eps, self.x_adv)
            self.x_adv = torch.clamp(self.x_adv, x_val_min, x_val_max)
            self.x_adv = Variable(self.x_adv.data, requires_grad=True)

        self.specs_from_xadv(print_info)
        

### Transfer Attack For Set Configuration

As a demo, we will perform a transfer attack from client 0 to the other 7 clients in the system. The implementation is identical to the FGSM testbed.

In [16]:
# Dummy bit of code, the transfer attacks don't happen properly without this for some reason

filename = 'exp3_neck2_2_head1'

# Generate Head and Neck NN objects
mode = 'cuda'
head_nn = CNN_Head(mode)
neck_nn = CNN_Neck(mode)

# Which network to load and directory
i = 0
exp_path = "Results/federated_system/"+filename+"/"
nn_path = exp_path + filename + "_"

# Load pre-trained weights
head_path = nn_path + str(i) +"_head_network"
neck_path = nn_path + str(i) +"_neck_network"

head = torch.load(head_path)
neck = torch.load(neck_path)
    
head_edit = OrderedDict()
neck_edit = OrderedDict()

# Edit the ordered_dict key names to be torch compatible
for key in head.keys():
    head_edit["network."+key] = head[key]

for key in neck.keys():
    neck_edit["network."+key] = neck[key]

head_nn.load_state_dict(head_edit)
neck_nn.load_state_dict(neck_edit)

/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


<All keys matched successfully>

In [17]:

# Load Config File and Slie Indices
with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
        
file_indices = [i for i in range(config['num_sets'])]
#random.shuffle(file_indices)
client_slice = len(file_indices)//config['num_clients']

# File names of FL trained setting
filenames = ["exp3_neck2_0_head3"]

# Matrix to Record Performance
orig_acc_transfers = np.zeros((1,config['num_clients']))
orig_similarities = np.zeros((1,config['num_clients']))
orig_target_hit = np.zeros((1,config['num_clients']))
adv_acc_transfers = np.zeros((1,config['num_clients']))
adv_similarities = np.zeros((1,config['num_clients']))
adv_target_hit = np.zeros((1,config['num_clients']))

# Attack Params
batch_size = 1000
iteration = 50
target = 5

# IFGSM
eps = 0.3
alpha = 0.3

# CWA
confidence = 0
optimizer_lr = 5e-4

In [18]:
# Basic loader for generating the victim

def load_victim(idx, loader):
    # Load the corresponding head/neck network in victim nn module 
    
    mode = 'cuda'
    #head_nn = CNN_Head(mode)
    #neck_nn = CNN_Neck(mode)
    
    # Which network to load and directory
    exp_path = "Results/federated_system/"+filename+"/"
    nn_path = exp_path + filename + "_"

    # Load pre-trained weights
    head_path = nn_path + str(idx) +"_head_network"
    neck_path = nn_path + str(idx) +"_neck_network"

    head = torch.load(head_path)
    neck = torch.load(neck_path)

    head_edit = OrderedDict()
    neck_edit = OrderedDict()

    # Edit the ordered_dict key names to be torch compatible
    for key in head.keys():
        head_edit["network."+key] = head[key]

    for key in neck.keys():
        neck_edit["network."+key] = neck[key]

    head_nn.load_state_dict(head_edit)
    neck_nn.load_state_dict(neck_edit)
    
    return Victim_NN(head_nn,neck_nn,loader)

Perform the transfer attack below.

In [19]:
for source in range(1):
    
    # Bring in the data loader for this client
    loader = Dataloader(file_indices,[source*(client_slice),min((source+1)*(client_slice),35)])  
    loader.load_training_dataset()
    loader.load_testing_dataset()

    victim_source = load_victim(source,loader)

    # Generate adversarial Perturbations
    victim_source.CW_attack(batch_size = batch_size, target= target, confidence=confidence, 
                            optimizer_lr=optimizer_lr, iteration=iteration, x_val_mean = (0.5), 
                            x_val_std= (0.5), print_info=False)

    # Record relevant tensors
    x_orig = victim_source.x_orig
    y_orig = victim_source.output_orig
    y_true = victim_source.y_orig
    x_adv = victim_source.x_adv
    y_adv = victim_source.output_adv

    print("======== Source", source, "========")

    for dest in range(config['num_clients']):

        print("    ==== Dest", dest, "====")

        victim_dest = load_victim(dest,loader)

        # Compute Stats and record
        victim_dest.forward_transfer(x_orig,x_adv,y_orig,y_adv,y_true, target, print_info=True)

        orig_acc_transfers[source,dest] = victim_dest.orig_test_acc
        orig_similarities[source,dest] = victim_dest.orig_output_sim
        orig_target_hit[source,dest] = victim_dest.orig_target_achieve

        adv_acc_transfers[source,dest] = victim_dest.adv_test_acc
        adv_similarities[source,dest] = victim_dest.adv_output_sim
        adv_target_hit[source,dest] = victim_dest.adv_target_achieve

Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json


NameError: name 'Victim_NN' is not defined